# Description

This notebook is used to tabulate final results and compare novel generated molecules vs HIV inhibitors vs Remdesivir which entered clinical trials just a few days ago.

## Now the best results and save to sdf for last double check tabulation in PyRx

In [2]:
import pandas as pd
from rdkit import Chem, DataStructs
import random
import numpy as np
import rdkit.Chem.PropertyMol

In [3]:
final = pd.read_csv('./generations/master_results_table_gen10.csv',sep=',')
final = final.sort_values('score', ascending=True)
print(final.shape)
final.head()

(3020, 6)


,id,gen,smile,source,weight,score
0,AABL,10,O=C(NC(Cc1ccccc1)C(=O)NC(Cc1ccccc1)C(=O)O)C1CC...,generated,786.926,-17.9
1,AADG,9,O=C(NC1CC2CCC(C1)N2CCc1ccccc1)C1CC2CC(C1)C2C(=...,generated,838.062,-17.7
2,AACA,10,O=C(NC1CC2CCC(C1)N2CCc1ccccc1)C1CC2CC(C1)C2C(=...,generated,838.062,-17.7
3,AABO,9,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C(...,generated,853.077,-17.6
4,AABK,10,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C(...,generated,853.077,-17.6


In [4]:
final_max = final.groupby("smile").max()[["score","gen"]].reset_index()
final_max = final_max.sort_values('score', ascending=True)
print(final_max.shape)
final_max.head()

(2728, 3)


,smile,score,gen
2437,O=C(NC(Cc1ccccc1)C(=O)NC(Cc1ccccc1)C(=O)O)C1CC...,-17.9,10
2474,O=C(NC1CC2CCC(C1)N2CCc1ccccc1)C1CC2CC(C1)C2C(=...,-17.7,10
2492,O=C(NC1CC2CCCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C...,-17.6,10
2482,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C(...,-17.6,10
2480,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CC(C1)C2C(=...,-17.5,10


In [5]:
final_joined = pd.merge(final_max, final, on=['smile','gen'], suffixes=('_old','_new'), how='left')
final_joined = final_joined[(final_joined['score_new'] <= -15.0) & (final_joined['weight'] < 900)]
final_joined['score'] = final_joined['score_new']
final_joined = final_joined.drop('score_old', axis=1)
final_joined = final_joined.drop('score_new', axis=1)
print(final_joined.shape)
final_joined.head()

(41, 6)


,smile,gen,id,source,weight,score
0,O=C(NC(Cc1ccccc1)C(=O)NC(Cc1ccccc1)C(=O)O)C1CC...,10,AABL,generated,786.926,-17.9
1,O=C(NC1CC2CCC(C1)N2CCc1ccccc1)C1CC2CC(C1)C2C(=...,10,AACA,generated,838.062,-17.7
2,O=C(NC1CC2CCCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C...,10,AABP,generated,867.104,-17.6
3,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C(...,10,AABK,generated,853.077,-17.6
4,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CC(C1)C2C(=...,10,AAAF,generated,839.050,-17.5


In [6]:
hiv = final[final['source'] == 'hiv']
hiv.head()

,id,gen,smile,source,weight,score
560,ABSI,0,CC(C)(C)NC(=O)C1CC2CCCCC2CN1CC(O)C(Cc1ccccc1)N...,hiv,670.855,-10.8
707,ABSH,0,CC(C)CN(CC(O)C(Cc1ccccc1)NC(=O)OC1COC2OCCC12)S...,hiv,756.004,-10.0
844,ABSN,0,COC(=O)NC(C(=O)NC(Cc1ccccc1)C(O)CN(Cc1ccc(-c2c...,hiv,704.869,-9.5
892,ABSK,0,CC(C)CN(CC(O)C(Cc1ccccc1)NC(=O)OC1COC2OCCC12)S...,hiv,547.674,-9.2
997,ABSL,0,Cc1c(O)cccc1C(=O)NC(CSc1ccccc1)C(O)CN1CC2CCCCC...,hiv,567.796,-8.5


In [7]:
final_table = final_joined.append(hiv)
final_table.head()

,smile,gen,id,source,weight,score
0,O=C(NC(Cc1ccccc1)C(=O)NC(Cc1ccccc1)C(=O)O)C1CC...,10,AABL,generated,786.926,-17.9
1,O=C(NC1CC2CCC(C1)N2CCc1ccccc1)C1CC2CC(C1)C2C(=...,10,AACA,generated,838.062,-17.7
2,O=C(NC1CC2CCCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C...,10,AABP,generated,867.104,-17.6
3,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CCC(C1)C2C(...,10,AABK,generated,853.077,-17.6
4,O=C(NC1CC2CCC(C1)N2CCc1ccccn1)C1CC2CC(C1)C2C(=...,10,AAAF,generated,839.050,-17.5


In [8]:
# Note I also manually added Remdesivir because as working on this it went into clinical trials
# https://en.wikipedia.org/wiki/Remdesivir
final_table.to_csv(r'./generations/master_results_table_final2.csv', index=False)

In [9]:
final_table = pd.read_csv('./generations/master_results_table_final.csv')
final_table.tail()

,smile,gen,id,source,weight,score_best,score_avg,similarity_to_hiv_inhibitors,similarity_to_remdesivir
50,CC(C)CN(CC(O)C(Cc1ccccc1)NC(=O)OC1CCOC1)S(=O)(...,0,ABSM,hiv,505.637,-7.7,-7.288889,0.558467,0.404068
51,CCCC1(CCc2ccccc2)CC(O)=C(C(CC)c2cccc(NS(=O)(=O...,0,ABSP,hiv,602.675,-7.6,-7.200000,0.493003,0.562334
52,O=C1Nc2ccc(Cl)cc2C(C#CC2CC2)(C(F)(F)F)O1,0,ABSQ,hiv,315.678,-6.4,-6.055556,0.433602,0.481928
53,CC(C)c1nc(CN(C)C(=O)NC(C(=O)NC(Cc2ccccc2)CC(O)...,0,ABSO,hiv,720.962,NaN,NaN,0.451159,0.476061
54,Cc1cccc(C)c1OCC(=O)NC(Cc1ccccc1)C(O)CC(Cc1cccc...,0,ABSJ,hiv,628.814,NaN,NaN,0.430199,0.422064


In [10]:
def set_molecule(row):
    mol = Chem.MolFromSmiles(row['smile'])
    pm = Chem.PropertyMol.PropertyMol(mol)
    title = 'id' + str(row['id']) + 'gen'+ str(row['gen'])
#     print(title)
    # Enables for tracking which molecule is which in PyRx GUI and PyRx results export
    pm.SetProp('Title', title)
    return pm

In [11]:
mols_for_export = final_table.apply(set_molecule, axis=1)
# list(mols_for_export)

In [12]:
w = Chem.SDWriter('./generations/genfinal.sdf')
for m in mols_for_export:
    w.write(m)

## Rerun everything through PyRx once more to double check, then publish final metrics

In [ ]:
new_scores = pd.read_csv('./generations/results/results_genfinal.csv',sep=',')
new_scores.head()

In [ ]:
new_scores = new_scores.groupby("Ligand").agg({'Binding Affinity': [np.min,np.average]}).reset_index()
new_scores.head()

In [ ]:
new_scores['id'] = new_scores['Ligand'].str.split("_").str[1].str.split("gen").str[0].str.split("id").str[1]
new_scores['gen'] = new_scores['Ligand'].str.split("_").str[1].str.split("gen").str[1]
new_scores['score_best'] = new_scores["Binding Affinity"]["amin"]
new_scores['score_avg'] = new_scores["Binding Affinity"]["average"]
new_scores = new_scores[['id','gen','score_best','score_avg']]
new_scores.head()

In [ ]:
new_scores.id = new_scores.id.astype(str)
new_scores.gen = new_scores.gen.astype(str)
final_table.id = final_table.id.astype(str)
final_table.gen = final_table.gen.astype(str)

In [ ]:
new_table = pd.merge(final_table, new_scores, on=['id','gen'], suffixes=('_old','_new'), how='left')
new_table['score_best'] = new_table[('score_best', '')]
new_table['score_avg'] = new_table[('score_avg', '')]
new_table = new_table.drop([('score_best', ''),('score_avg', '')], axis=1)
new_table = new_table.sort_values('score_best', ascending=True)
new_table.tail()

In [ ]:
new_table.columns

In [ ]:
hiv_smiles = new_table[(new_table['source'] == 'hiv') & (new_table['score_best'].notnull())]
hiv_smiles_list = list(hiv_smiles['smile'])
hiv_smiles.head(20)

In [ ]:
hiv_fingerprints = []
for smile in hiv_smiles_list:
    hiv_fingerprints.append(Chem.RDKFingerprint(Chem.MolFromSmiles(smile)))

def calc_hiv_similarity_score(row):
    fingerprint = Chem.RDKFingerprint(Chem.MolFromSmiles(row['smile']))
    similarity = np.mean(DataStructs.BulkTanimotoSimilarity(fingerprint,hiv_fingerprints))
    return similarity

remdesivir_fingerprint = Chem.RDKFingerprint(Chem.MolFromSmiles('CCC(CC)COC(=O)[C@H](C)NP(=O)(OC[C@H]1O[C@](C#N)([C@H](O)[C@@H]1O)C1=CC=C2N1N=CN=C2N)OC1=CC=CC=C1'))

new_table['similarity_to_hiv_inhibitors'] = new_table.apply(calc_hiv_similarity_score, axis=1)
new_table['similarity_to_remdesivir'] = new_table['smile'].apply(lambda x: DataStructs.TanimotoSimilarity(Chem.RDKFingerprint(Chem.MolFromSmiles(x)),remdesivir_fingerprint))
new_table.head(50)

In [ ]:
new_table.to_csv(r'./generations/master_results_table_final.csv', index=False)